# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fdccc49fbb0>
├── 'a' --> tensor([[-0.4667, -0.4830, -0.0972],
│                   [ 0.7158, -1.7944,  1.3706]])
└── 'x' --> <FastTreeValue 0x7fdccc492370>
    └── 'c' --> tensor([[ 0.6759, -0.4700, -0.1132, -1.5012],
                        [-0.4585,  1.8002, -0.2085, -0.2673],
                        [-1.1971, -1.4469,  1.7908, -0.1042]])

In [4]:
t.a

tensor([[-0.4667, -0.4830, -0.0972],
        [ 0.7158, -1.7944,  1.3706]])

In [5]:
%timeit t.a

63.8 ns ± 0.0398 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fdccc49fbb0>
├── 'a' --> tensor([[0.2326, 0.3944, 1.0796],
│                   [1.6243, 1.0636, 0.7587]])
└── 'x' --> <FastTreeValue 0x7fdccc492370>
    └── 'c' --> tensor([[ 0.6759, -0.4700, -0.1132, -1.5012],
                        [-0.4585,  1.8002, -0.2085, -0.2673],
                        [-1.1971, -1.4469,  1.7908, -0.1042]])

In [7]:
%timeit t.a = new_value

68.5 ns ± 0.0827 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4667, -0.4830, -0.0972],
               [ 0.7158, -1.7944,  1.3706]]),
    x: Batch(
           c: tensor([[ 0.6759, -0.4700, -0.1132, -1.5012],
                      [-0.4585,  1.8002, -0.2085, -0.2673],
                      [-1.1971, -1.4469,  1.7908, -0.1042]]),
       ),
)

In [10]:
b.a

tensor([[-0.4667, -0.4830, -0.0972],
        [ 0.7158, -1.7944,  1.3706]])

In [11]:
%timeit b.a

59.8 ns ± 0.0309 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0485,  0.1640, -1.2017],
               [ 0.0734, -0.9911, -0.2694]]),
    x: Batch(
           c: tensor([[ 0.6759, -0.4700, -0.1132, -1.5012],
                      [-0.4585,  1.8002, -0.2085, -0.2673],
                      [-1.1971, -1.4469,  1.7908, -0.1042]]),
       ),
)

In [13]:
%timeit b.a = new_value

500 ns ± 0.182 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

806 ns ± 0.845 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 17.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 809 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

147 µs ± 2.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fdc22162340>
├── 'a' --> tensor([[[-0.4667, -0.4830, -0.0972],
│                    [ 0.7158, -1.7944,  1.3706]],
│           
│                   [[-0.4667, -0.4830, -0.0972],
│                    [ 0.7158, -1.7944,  1.3706]],
│           
│                   [[-0.4667, -0.4830, -0.0972],
│                    [ 0.7158, -1.7944,  1.3706]],
│           
│                   [[-0.4667, -0.4830, -0.0972],
│                    [ 0.7158, -1.7944,  1.3706]],
│           
│                   [[-0.4667, -0.4830, -0.0972],
│                    [ 0.7158, -1.7944,  1.3706]],
│           
│                   [[-0.4667, -0.4830, -0.0972],
│                    [ 0.7158, -1.7944,  1.3706]],
│           
│                   [[-0.4667, -0.4830, -0.0972],
│                    [ 0.7158, -1.7944,  1.3706]],
│           
│                   [[-0.4667, -0.4830, -0.0972],
│                    [ 0.7158, -1.7944,  1.3706]]])
└── 'x' --> <FastTreeValue 0x7fdc22162760>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.9 µs ± 56 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fdccc50c5b0>
├── 'a' --> tensor([[-0.4667, -0.4830, -0.0972],
│                   [ 0.7158, -1.7944,  1.3706],
│                   [-0.4667, -0.4830, -0.0972],
│                   [ 0.7158, -1.7944,  1.3706],
│                   [-0.4667, -0.4830, -0.0972],
│                   [ 0.7158, -1.7944,  1.3706],
│                   [-0.4667, -0.4830, -0.0972],
│                   [ 0.7158, -1.7944,  1.3706],
│                   [-0.4667, -0.4830, -0.0972],
│                   [ 0.7158, -1.7944,  1.3706],
│                   [-0.4667, -0.4830, -0.0972],
│                   [ 0.7158, -1.7944,  1.3706],
│                   [-0.4667, -0.4830, -0.0972],
│                   [ 0.7158, -1.7944,  1.3706],
│                   [-0.4667, -0.4830, -0.0972],
│                   [ 0.7158, -1.7944,  1.3706]])
└── 'x' --> <FastTreeValue 0x7fdccc50c190>
    └── 'c' --> tensor([[ 0.6759, -0.4700, -0.1132, -1.5012],
                        [-0.4585,  1.8002, -0.2085, -0.2673],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 41.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.7 µs ± 81.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.4667, -0.4830, -0.0972],
                [ 0.7158, -1.7944,  1.3706]],
       
               [[-0.4667, -0.4830, -0.0972],
                [ 0.7158, -1.7944,  1.3706]],
       
               [[-0.4667, -0.4830, -0.0972],
                [ 0.7158, -1.7944,  1.3706]],
       
               [[-0.4667, -0.4830, -0.0972],
                [ 0.7158, -1.7944,  1.3706]],
       
               [[-0.4667, -0.4830, -0.0972],
                [ 0.7158, -1.7944,  1.3706]],
       
               [[-0.4667, -0.4830, -0.0972],
                [ 0.7158, -1.7944,  1.3706]],
       
               [[-0.4667, -0.4830, -0.0972],
                [ 0.7158, -1.7944,  1.3706]],
       
               [[-0.4667, -0.4830, -0.0972],
                [ 0.7158, -1.7944,  1.3706]]]),
    x: Batch(
           c: tensor([[[ 0.6759, -0.4700, -0.1132, -1.5012],
                       [-0.4585,  1.8002, -0.2085, -0.2673],
                       [-1.1971, -1.4469,  1.7908, -0.1042]],
         

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.4667, -0.4830, -0.0972],
               [ 0.7158, -1.7944,  1.3706],
               [-0.4667, -0.4830, -0.0972],
               [ 0.7158, -1.7944,  1.3706],
               [-0.4667, -0.4830, -0.0972],
               [ 0.7158, -1.7944,  1.3706],
               [-0.4667, -0.4830, -0.0972],
               [ 0.7158, -1.7944,  1.3706],
               [-0.4667, -0.4830, -0.0972],
               [ 0.7158, -1.7944,  1.3706],
               [-0.4667, -0.4830, -0.0972],
               [ 0.7158, -1.7944,  1.3706],
               [-0.4667, -0.4830, -0.0972],
               [ 0.7158, -1.7944,  1.3706],
               [-0.4667, -0.4830, -0.0972],
               [ 0.7158, -1.7944,  1.3706]]),
    x: Batch(
           c: tensor([[ 0.6759, -0.4700, -0.1132, -1.5012],
                      [-0.4585,  1.8002, -0.2085, -0.2673],
                      [-1.1971, -1.4469,  1.7908, -0.1042],
                      [ 0.6759, -0.4700, -0.1132, -1.5012],
                      [-0.4585,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

329 µs ± 2.78 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
